In [86]:
#Important notes: All those greater than 5 had consensus.
import numpy as np
from IPython.core.display import HTML
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## THIS IS THE FINAL JUN 8 at 3
finaltaskusdf = pd.DataFrame()
finallocdf = pd.DataFrame()
data = {}
all_df= []
df_s = pd.DataFrame()
files =  ['au-extracted.json','it-extracted.json', 
          'fr-extracted.json','gb-extracted.json',
             'ca-extracted.json', 'us-extracted.json']
              

for file_num, file_name in enumerate(files):
    data = {}
    print(str(file_name))
    n=0
    with open('/Users/kimia/Desktop/Work5/sprint614/' + str(file_name)) as f:
            
                for line in f:
                    try:

                        data.update(json.loads( line.rstrip('\n')))
                        all_df.append(data)

                        taskusedits = pd.Series(data['data']['extraction']['payload']).to_frame().T
                        taskusedits['id'] = (data['uri'])
                        taskusedits['file'] = str(file_name)

                        locdf = pd.Series(data['data']['extraction']['rawPayload']).to_frame().T
                        locdf['id'] = (data['uri'])
                        locdf['file'] = str(file_name)


                        finaltaskusdf = pd.concat([taskusedits, finaltaskusdf], axis=0)
                        finallocdf = pd.concat([locdf, finallocdf], axis=0)
       
                    except:

                        n = n+1
                        print(n)
finallocdf['id'] = finallocdf['id'].apply(lambda x: int(x[-6:]))
finaltaskusdf['id']= finaltaskusdf['id'].apply(lambda x: int(x[-6:]))

## THIS WORK!! 1:36 JUN 7
data = {}
df_s = pd.DataFrame()
files = ['ca-seed-inputs.json' ,
             'it-seed-inputs.json', 'us-seed-inputs.json', 'au-seed-inputs.json', 
             'gb-seed-inputs.json' , 
             'fr-seed-inputs.json']
for file_num, file_name in enumerate(files):

    print(str(file_name)+"  i")
    with open('/Users/kimia/Desktop/Work5/sprint614/' + str(file_name)) as f:
        
        
        for line in f:
            data.update(json.loads( line.rstrip('\n')))
            #print(data['data']['extraction']['inputMeta'])

            d1 = pd.Series(data['data']['extraction']['inputMeta']).to_frame().T 
        
            d1['file'] = str(file_name)
            df_s = pd.concat([df_s, d1], axis=0)
        
result = (pd.merge(df_s,finaltaskusdf, on = ['id'])) # correct merge: (result.file_x == result.file_y)

from urllib.parse import urlparse
result['cleansite'] =result['website'].apply(lambda x: urlparse(str(x)).netloc)
result['cleansite'] =result['cleansite'].apply(lambda x: (str(x).replace("www.", "")))

au-extracted.json
it-extracted.json
fr-extracted.json
1
gb-extracted.json
1
2
3
4
5
ca-extracted.json
1
2
us-extracted.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
ca-seed-inputs.json  i
it-seed-inputs.json  i
us-seed-inputs.json  i
au-seed-inputs.json  i
gb-seed-inputs.json  i
fr-seed-inputs.json  i


In [3]:
dfresult = result
def consensus(df, column):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] >4]
    perc5 = (len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy  

columnsz = [
 'address',
 'name',
 'tel',
 'cleansite']

final = pd.DataFrame()
accuracydict={}
for number,country in enumerate(result.file_x.unique()):
    
    for i, v in enumerate(columnsz): 


        percent = (consensus(dfresult[dfresult.file_x ==str(country)], str(v)))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: % there is consensus among 5 on a POI, by field input")
display(HTML(((final).to_html())))


    MODERATION: % there is consensus among 5 on a POI, by field input


,fr,gb,au,us,it,ca
address,0.442029,0.320896,0.636842,0.635514,0.416667,0.492537
cleansite,0.536232,0.664179,0.747368,0.623053,0.437500,0.686567
name,0.710145,0.783582,0.831579,0.800623,0.562500,0.716418
tel,0.746377,0.686567,0.768421,0.641745,0.463542,0.776119


In [158]:
result['cleanAddress'] =result['address'].apply(lambda x: (str(x)).lower())
result['cleanname'] =result['name'].apply(lambda x: (str(x)).lower())

dfresult = result
def consensus(df, column):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] >4]
    perc5 = (len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy  

columnsz = [
 'cleanAddress', 
 'cleanname',
 'tel',
 'cleansite']

final = pd.DataFrame()
accuracydict={}
for number,country in enumerate(result.file_x.unique()):
    
    for i, v in enumerate(columnsz): 


        percent = (consensus(dfresult[dfresult.file_x ==str(country)], str(v)))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: % there is consensus among 5 on a POI, by field input")
display(HTML(((final).to_html())))


    MODERATION: % there is consensus among 5 on a POI, by field input


,fr,gb,au,us,it,ca
cleanAddress,0.442029,0.320896,0.636842,0.635514,0.427083,0.552239
cleanname,0.717391,0.791045,0.831579,0.800623,0.567708,0.731343
cleansite,0.536232,0.664179,0.747368,0.623053,0.437500,0.686567
tel,0.736842,0.686567,0.767196,0.640625,0.434066,0.761905


In [96]:
datafr =result   
datafr = datafr.groupby('task_id').size().to_frame().reset_index()
print(len((datafr[datafr[0]!=5]).task_id.unique()))
datafr = (datafr[datafr[0]==5])
print(len(datafr.task_id.unique()))

datafr = pd.merge(datafr,result, on ='task_id',  how = 'left')
print(len(ff.task_id.unique()))

30
1012
1012


In [115]:
def consensus(df, column, samplesize):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] == samplesize]
    perc5 = (len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy  






columnsz = [
 'address',
 'name',
 'tel',
 'cleansite']

samplesize = 3
datafr =result   
datafr = datafr.groupby('task_id').size().to_frame().reset_index()
datafr = (datafr[datafr[0]==5])
datafr = pd.merge(datafr,result, on ='task_id',  how = 'left')


final = pd.DataFrame()
accuracydict={}

for number,country in enumerate(result.file_x.unique()):

    for i, v in enumerate(columnsz): 
        df2 = datafr[datafr.file_x == str(country)]
        dfsample4 = df2.groupby(['task_id']).apply(lambda x: (x.sample(int(samplesize)))).reset_index(drop=True)


        percent = (consensus(dfsample4, str(v), samplesize))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
final['samplesize'] = samplesize
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: % there is consensus")
final3 = final
display(HTML(((final).to_html())))

    MODERATION: % there is consensus


,fr,gb,au,us,it,ca,samplesize
address,0.560345,0.284314,0.728814,0.618852,0.511628,0.557377,3
cleansite,0.598485,0.775194,0.801075,0.703822,0.529730,0.727273,3
name,0.811024,0.829268,0.887097,0.856230,0.654054,0.784615,3
tel,0.793651,0.742188,0.810811,0.726688,0.569767,0.822581,3


In [124]:
final5

,fr,gb,au,us,it,ca,samplesize
address,0.391667,0.207207,0.620112,0.547244,0.394286,0.460317,5
cleansite,0.545455,0.674419,0.747312,0.617834,0.443243,0.681818,5
name,0.708661,0.780488,0.827957,0.798722,0.572973,0.707692,5
tel,0.748031,0.689922,0.762162,0.638978,0.440000,0.758065,5


In [123]:
print('      consensus among attribute values, relative to annotator sample size')
pd.concat([final5,final4,final3]).reset_index().sort_values(['index'])

      consensus among attribute values, relative to annotator sample size


,index,fr,gb,au,us,it,ca,samplesize
0,address,0.391667,0.207207,0.620112,0.547244,0.394286,0.460317,5
4,address,0.453782,0.281818,0.657303,0.580000,0.459770,0.492063,4
8,address,0.560345,0.284314,0.728814,0.618852,0.511628,0.557377,3
1,cleansite,0.545455,0.674419,0.747312,0.617834,0.443243,0.681818,5
5,cleansite,0.583333,0.689922,0.768817,0.652866,0.486486,0.712121,4
9,cleansite,0.598485,0.775194,0.801075,0.703822,0.529730,0.727273,3
2,name,0.708661,0.780488,0.827957,0.798722,0.572973,0.707692,5
6,name,0.740157,0.804878,0.838710,0.824281,0.621622,0.753846,4
10,name,0.811024,0.829268,0.887097,0.856230,0.654054,0.784615,3
3,tel,0.748031,0.689922,0.762162,0.638978,0.440000,0.758065,5


In [151]:
pd.concat([final5,final4,final3]).reset_index().sort_values(['index'])
dff = pd.concat([final5,final3]).reset_index().sort_values(['index'])
dff = dff.T
dff
dff = dff[1:7]
dff['address'] = dff[0]-dff[4]
dff['website'] = dff[1]-dff[5]
dff['name'] = dff[2]-dff[6]
dff['tel'] = dff[3]-dff[7]
dff = (dff*-1)*100
print('% consensus gained by reducing \n annotation from 5 to 3')
dff[['address', 'website', 'name', 'tel']]

% consensus gained by reducing 
 annotation from 5 to 3


,address,website,name,tel
fr,16.8678,5.30303,10.2362,4.56193
gb,7.71065,10.0775,4.87805,5.2265
au,10.8702,5.37634,5.91398,4.86486
us,7.16084,8.59873,5.7508,8.77105
it,11.7342,8.64865,8.10811,12.9767
ca,9.70596,4.54545,7.69231,6.45161


In [ ]:
def consensus(df, column, samplesize):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] == samplesize]
    perc5 = (len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy  


columnsz = [
 'address',
 'name',
 'tel',
 'cleansite']

samplesize = 2
df= {}

## set up frame so no sample error
datafr =result   
datafr = datafr.groupby('task_id').size().to_frame().reset_index()
datafr = (datafr[datafr[0]==5])
datafr = pd.merge(datafr,result, on ='task_id',  how = 'left')

final = pd.DataFrame()
final2= pd.DataFrame()
accuracydict={}
#for numah, samplesize in enumerate([2,3,4,5]):
for number,country in enumerate(result.file_x.unique()):

    for i, v in enumerate(columnsz): 
        df2 = datafr[datafr.file_x == str(country)]
        dfsample4 = df2.groupby(['task_id']).apply(lambda x: (x.sample(int(samplesize)))).reset_index(drop=True)


        percent = (consensus(dfsample4, str(v), samplesize))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #final2 = pd.concat([final, final2], axis =0)
    #final['samplesize'] = samplesize
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: % there is consensus")
display(HTML(((final2).to_html())))

In [102]:
display(HTML(((final2).to_html())))

,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca,fr,gb,au,us,it,ca
address,0.391667,0.207207,0.620112,0.547244,0.394286,0.460317,0.461538,0.250000,0.648045,0.587755,0.451429,0.476190,0.495726,0.364583,0.738636,0.637500,0.522989,0.548387,0.710526,0.451613,0.777143,0.706383,0.610465,0.698413,0.461538,0.250000,0.648045,0.587755,0.451429,0.476190,0.495726,0.364583,0.738636,0.637500,0.522989,0.548387,0.710526,0.451613,0.777143,0.706383,0.610465,0.698413,0.495726,0.364583,0.738636,0.637500,0.522989,0.548387,0.710526,0.451613,0.777143,0.706383,0.610465,0.698413,0.710526,0.451613,0.777143,0.706383,0.610465,0.698413
cleansite,0.545455,0.674419,0.747312,0.617834,0.443243,0.681818,0.568182,0.689922,0.784946,0.649682,0.486486,0.712121,0.621212,0.775194,0.795699,0.710191,0.545946,0.742424,0.712121,0.798450,0.849462,0.802548,0.675676,0.787879,0.568182,0.689922,0.784946,0.649682,0.486486,0.712121,0.621212,0.775194,0.795699,0.710191,0.545946,0.742424,0.712121,0.798450,0.849462,0.802548,0.675676,0.787879,0.621212,0.775194,0.795699,0.710191,0.545946,0.742424,0.712121,0.798450,0.849462,0.802548,0.675676,0.787879,0.712121,0.798450,0.849462,0.802548,0.675676,0.787879
name,0.708661,0.780488,0.827957,0.798722,0.572973,0.707692,0.763780,0.796748,0.849462,0.827476,0.610811,0.723077,0.771654,0.853659,0.881720,0.855769,0.697297,0.800000,0.857143,0.886179,0.908602,0.888179,0.751351,0.861538,0.763780,0.796748,0.849462,0.827476,0.610811,0.723077,0.771654,0.853659,0.881720,0.855769,0.697297,0.800000,0.857143,0.886179,0.908602,0.888179,0.751351,0.861538,0.771654,0.853659,0.881720,0.855769,0.697297,0.800000,0.857143,0.886179,0.908602,0.888179,0.751351,0.861538,0.857143,0.886179,0.908602,0.888179,0.751351,0.861538
tel,0.748031,0.689922,0.762162,0.638978,0.440000,0.758065,0.769841,0.710938,0.772973,0.686901,0.465517,0.806452,0.840000,0.716535,0.816216,0.714744,0.500000,0.854839,0.848000,0.795276,0.859459,0.785714,0.635294,0.854839,0.769841,0.710938,0.772973,0.686901,0.465517,0.806452,0.840000,0.716535,0.816216,0.714744,0.500000,0.854839,0.848000,0.795276,0.859459,0.785714,0.635294,0.854839,0.840000,0.716535,0.816216,0.714744,0.500000,0.854839,0.848000,0.795276,0.859459,0.785714,0.635294,0.854839,0.848000,0.795276,0.859459,0.785714,0.635294,0.854839


In [ ]:
all_probabilities = pd.DataFrame()

#probabilities of 4
for i,sample in enumerate([2,3,4,5]):
    df = dfpayloads[dfpayloads['annotation']!= "unsure"]

    df['unique'] = (df.task_id).apply(lambda x: str(x)) + (df.payload).apply(lambda x: str(x))

    df2 = df.groupby('unique').size().to_frame().reset_index()
    df2 = (df2[df2[0]>4])
    df5 = pd.merge(df2, df, on = 'unique')


    ## take a sample
    dfsample4 = df5.groupby('unique').apply(lambda x: (x.sample(int(sample)))).reset_index(drop=True)
    # make duplicate means 1 if dup, else = 0
    dfsample4["dup"] = 0
    dfsample4.loc[dfsample4['annotation'] == 'duplicate', 'dup'] = 1
    # get the ratio of consensus
    prob = dfsample4.groupby('unique').apply(lambda x: (np.sum(x['dup'])/int(sample))).reset_index(drop=True)

    # only where everyone totally agrees. 
    probabilities = pd.DataFrame(prob)
    probabilities['sample'] = int(sample)
    probabilities['consensus'] = 0
    probabilities.loc[probabilities[0] == 0, 'consensus'] = 1
    probabilities.loc[probabilities[0] == 1, 'consensus'] = 1
    all_probabilities = pd.concat([all_probabilities, probabilities], axis = 0)

